In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# read in the csv files
portfolio = pd.read_csv('data/portfolio_clean.csv')
profile = pd.read_csv('data/profile_clean.csv')
transcript = pd.read_csv('data/transcript_clean.csv')

del portfolio['Unnamed: 0']
del profile['Unnamed: 0']
del transcript['Unnamed: 0']

In [3]:
sql_connect = sqlite3.connect('starbucks.db')
cursor = sql_connect.cursor()

## Save to tables in database

In [4]:
portfolio = portfolio.set_index('id')
profile = profile.set_index('id')

In [5]:
portfolio.to_sql('portfolio', sql_connect, if_exists='replace')
profile.to_sql('profile', sql_connect, if_exists='replace')
transcript.to_sql('transcript', sql_connect, if_exists='replace')

## Create target label
If offer was received and viewed, label = 1; otherwise 0

In [6]:
query = """
SELECT DISTINCT a.person, a.offer_id, a.time as received_time, b.time as viewed_time
, CASE WHEN b.person IS NULL THEN 0 ELSE 1 END as label
FROM (SELECT person, offer_id, time 
FROM transcript
WHERE event = 'offer received') a
LEFT JOIN (SELECT person, offer_id, time
FROM transcript
WHERE event = 'offer viewed') b
    ON a.person = b.person
    AND a.offer_id = b.offer_id
    AND a.time <= b.time
"""

In [7]:
pd.read_sql_query(query,sql_connect).to_sql('transcript_rec_view', sql_connect, if_exists='replace')

For customers receiving and viewing the same offers, use the min view time as the only view time for that offer

In [8]:
query = """
SELECT a.person, a.offer_id, a.received_time, a.viewed_time-a.received_time as duration_view, label
FROM (SELECT person, offer_id, received_time, min(viewed_time) as viewed_time, max(label) as label
FROM transcript_rec_view
GROUP BY person, offer_id, received_time) a
"""

In [9]:
transcript_final = pd.read_sql_query(query,sql_connect)
transcript_final.to_sql('transcript_final', sql_connect, if_exists='replace')

In [10]:
assert sum(transcript['event'] == 'offer received') == transcript_final.shape[0]\
, "Incorrect dimension - the number of 'offer received' in original transcript should equal to transcript_final"

## Create additional offer and customer features based on transcript

### Split into training and testing dataset

In [11]:
# To prevent data leakage when creating features based on transcript
# if the time is smaller than a threshold, make it training data and larger than a threshold as testing data
# training : testing should roughly be 3:1

transcript_quantile = transcript_final.groupby('person')['received_time'].quantile(0.75).reset_index()
transcript_quantile.to_sql('transcript_quantile', sql_connect, if_exists='replace')

In [12]:
query = """
SELECT a.*, CASE WHEN a.received_time <= b.received_time THEN 1 ELSE 0 END as training_label
FROM transcript_final a
LEFT JOIN transcript_quantile b
        ON a.person = b.person    
"""

In [13]:
assert pd.read_sql_query(query,sql_connect).shape[0]==transcript_final.shape[0] \
, "Wrong data dimension after joining with quantile"

In [14]:
transcript_final = pd.read_sql_query(query,sql_connect)
transcript_final.to_sql('transcript_final', sql_connect, if_exists='replace')

In [15]:
assert transcript_final['training_label'].sum()/ transcript_final.shape[0] > 0.7 \
, "Training data accounts for less than 70% of the total data"

In [16]:
transcript_training = transcript_final[transcript_final['training_label']==1]

In [17]:
transcript_testing = transcript_final[transcript_final['training_label']==0]

In [18]:
assert len(transcript_training['person'].unique()) >= len(transcript_testing['person'].unique()) \
, "Training data include fewer customer than in the testin data"

In [19]:
transcript_training.to_sql('transcript_training', sql_connect, if_exists='replace')
transcript_testing.to_sql('transcript_testing', sql_connect, if_exists='replace')

### Create offer features (only use transcript_training)

In [20]:
def create_features_using_groupby(df, entity_col, feature, avg=True, min=True, max=True):
    groupby = transcript_training.groupby(entity_col)[feature]
    
    features, col_name = [], []
    if avg:
        features.append(groupby.mean())
        col_name.append('avg_'+feature)
    if min:
        features.append(groupby.min())
        col_name.append('min_'+feature)
    if max:
        features.append(groupby.max())
        col_name.append('max_'+feature)
        
    feature_df = pd.concat(features, axis=1)
    feature_df.columns = col_name
    
    return feature_df

In [21]:
portfolio_duration = create_features_using_groupby(transcript_testing, 'offer_id', 'duration_view')

In [22]:
portfolio_view_rate = create_features_using_groupby(transcript_testing, 'offer_id', 'label', min=False, max=False)
portfolio_view_rate.columns=['view_rate']

In [23]:
portfolio_feat = pd.concat([portfolio_view_rate, portfolio_duration], axis=1)

#### check dimension match

In [24]:
assert portfolio_feat.shape[0] == portfolio.shape[0], "rows do not match with original data (portfolio)"

In [25]:
portfolio = portfolio.join(portfolio_feat)

#### save to database

In [26]:
portfolio.to_sql('portfolio', sql_connect, if_exists='replace')

### Create customer features (only use transcript_training)

In [27]:
query = """
SELECT a.person, min(amount) as min_amount, max(amount) as max_amount, avg(amount) as avg_amount
FROM transcript a
    JOIN transcript_quantile b
        ON a.person = b.person  
WHERE a.time <= b.received_time
GROUP BY a.person
"""

In [28]:
profile_amount = pd.read_sql_query(query,sql_connect).set_index('person')

In [29]:
profile_duration = create_features_using_groupby(transcript_testing, 'person', 'duration_view')

In [30]:
profile_view_rate = create_features_using_groupby(transcript_testing, 'person', 'label', min=False, max=False)
profile_view_rate.columns=['view_rate']

In [31]:
profile_trx_rate = (transcript_training.groupby('person').size()*100/(transcript_training.groupby('person')['received_time'].max() - transcript_training.groupby('person')['received_time'].min())).reset_index()
profile_trx_rate.columns = ['person', 'avg_trx_cnt']
profile_trx_rate.loc[profile_trx_rate['avg_trx_cnt']==np.inf, 'avg_trx_cnt'] = 1
profile_trx_rate = profile_trx_rate.set_index('person')

In [32]:
profile_feat = profile_amount.join(profile_duration).join(profile_view_rate).join(profile_trx_rate)

#### check dimension match

In [33]:
assert pd.merge(profile, profile_feat, how='left', left_index=True, right_index=True).shape[0] == profile.shape[0], "rows do not match with original data (profile)"

In [34]:
profile = pd.merge(profile, profile_feat, how='left', left_index=True, right_index=True)

#### save to database

In [35]:
profile.to_sql('profile', sql_connect, if_exists='replace')

In [36]:
cursor.close()